#task #1: 問題提起とビジネスケースを理解する

![alt text](https://drive.google.com/uc?id=1djxup79_KiGtKFiH7AgSD0Bj-2D90TBg)

#task #2: ライブラリ/データセットのインポートと探索的データ分析の実行

In [ ]:
# import key libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import plotly.express as px

# Tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

In [ ]:
# Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# install nltk
# NLTK: Natural Language tool kit
!pip install nltk

In [ ]:
# install gensim
# Gensimは、教師なしのトピックモデリングと自然言語処理のためのオープンソースのライブラリです。
# Gensim は Python と Cython で実装されています。

!pip install gensim

In [ ]:
# 株式ニュースデータの読み込み
stock_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Python & ML in Finance/stock_sentiment.csv")

In [ ]:
# データセットを表示してみよう 
stock_df

In [ ]:
# データフレームの情報
stock_df.info()

In [ ]:
# null値のチェック
stock_df.isnull().sum()

**ミニ・チャレンジ #1:**

- **「sentiment」列にはユニークな要素がいくつありますか？2つの異なる方法で調べてみてください。


#task #3: データクリーニングを行う (テキストから句読点を取り除く)

In [ ]:
import string
string.punctuation

In [ ]:
Test = '$I love AI & Machine learning!!'
Test_punc_removed = [char for char in Test if char not in string.punctuation]
Test_punc_removed_join = ''.join(Test_punc_removed)
Test_punc_removed_join

In [ ]:
Test = 'Good morning beautiful people :)... #I am having fun learning Finance with Python!!'

In [ ]:
Test_punc_removed = [char for char in Test if char not in string.punctuation]
Test_punc_removed

In [ ]:
# 再び文字を結合して文字列を形成する。
Test_punc_removed_join = ''.join(Test_punc_removed)
Test_punc_removed_join

In [ ]:
# 句読点を除去する関数を定義しよう
def remove_punc(message):
    Test_punc_removed = [char for char in message if char not in string.punctuation]
    Test_punc_removed_join = ''.join(Test_punc_removed)

    return Test_punc_removed_join

In [ ]:
# データセットから句読点を削除してみよう 
stock_df['Text Without Punctuation'] = stock_df['Text'].apply(remove_punc)

In [ ]:
stock_df

In [ ]:
stock_df['Text'][2]

In [ ]:
stock_df['Text Without Punctuation'][2]

**ミニチャレンジ #2:** 
- **別の方法で句読点を削除する**。


# task #4: データクリーニング（ストップワードの削除）を行う

In [ ]:
# ストップワードのダウンロード
nltk.download("stopwords")
stopwords.words('english')

In [ ]:
# nltkから追加のストップワードを取得する
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'will', 'aap', 'co', 'day', 'user', 'stock', 'today', 'week', 'year'])
# stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'will', 'aap', 'co', 'day', 'user', 'stock','today', 'week', 'year', 'https'])

In [ ]:
# ストップワードの除去と短い単語（2文字以下）の除去
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if len(token) >= 3 and token not in stop_words:
            result.append(token)
            
    return result

In [ ]:
# text列に前処理を施す
stock_df['Text Without Punc & Stopwords'] = stock_df['Text Without Punctuation'].apply(preprocess)

In [ ]:
stock_df['Text'][0]

In [ ]:
stock_df['Text Without Punc & Stopwords'][0]

In [ ]:
# 単語を文字列に結合
# stock_df['Processed Text 2'] = stock_df['Processed Text 2'].apply(lambda x: " ".join(x))


In [ ]:
stock_df

**ミニチャレンジ #3:**

- **3文字ではなく2文字以上の単語を保持するようにコードを修正する**。
- **ストップワードのリストに'https'を追加し、コードを再実行する**。


# task #5: plot wordcloud

In [ ]:
# 単語を文字列に結合する
stock_df['Text Without Punc & Stopwords Joined'] = stock_df['Text Without Punc & Stopwords'].apply(lambda x: " ".join(x))

In [ ]:
# 肯定的な感情を持つテキストのワードクラウドをプロットする
plt.figure(figsize = (20, 20)) 
wc = WordCloud(max_words = 1000 , width = 1600 , height = 800).generate(" ".join(stock_df[stock_df['Sentiment'] == 1]['Text Without Punc & Stopwords Joined']))
plt.imshow(wc, interpolation = 'bilinear');

**ミニチャレンジ#4:**
- **ネガティブな感情を持つツイートのワードクラウドを可視化する**。


# task #6: クリーニングされたデータセットの可視化

In [ ]:
stock_df

In [ ]:
nltk.download('punkt')

In [ ]:
# word_tokenizeは、文字列を単語に分解するために使います
print(stock_df['Text Without Punc & Stopwords Joined'][0])
print(nltk.word_tokenize(stock_df['Text Without Punc & Stopwords Joined'][0]))

In [ ]:
# ドキュメント内のデータの最大長を取得します。
# これは後で単語埋め込みを生成する際に使用される

maxlen = -1
for doc in stock_df['Text Without Punc & Stopwords Joined']:
    tokens = nltk.word_tokenize(doc)
    if(maxlen < len(tokens)):
        maxlen = len(tokens)
print("The maximum number of words in any document is:", maxlen)

In [ ]:
tweets_length = [ len(nltk.word_tokenize(x)) for x in stock_df['Text Without Punc & Stopwords Joined'] ]
tweets_length

In [ ]:
# テキスト中の単語数の分布をプロットする
fig = px.histogram(x = tweets_length, nbins = 50)
fig.show()

**ミニチャレンジ #5:**
- **Seaborn Countplotを使って、肯定的な感情と否定的な感情のクラスに属するサンプルの数を視覚的に示してください**。


# task #7: トークン化とパディングによるデータの準備

![alt text](https://drive.google.com/uc?id=13j8m-JOpK994CtukR1EShiY_hGGjkNx-)

In [ ]:
stock_df

In [ ]:
# データセットに含まれる単語の総数を得る
list_of_words = []
for i in stock_df['Text Without Punc & Stopwords']:
    for j in i:
        list_of_words.append(j)


In [ ]:
list_of_words

In [ ]:
# ユニークな単語の総数を得る
total_words = len(list(set(list_of_words)))
total_words

In [ ]:
# データをテストとトレーニングに分ける 
X = stock_df['Text Without Punc & Stopwords']
y = stock_df['Sentiment']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train

In [ ]:
# 単語をトークン化するトークナイザーを作成し，トークン化された単語のシーケンスを作成する
tokenizer = Tokenizer(num_words = total_words)
tokenizer.fit_on_texts(X_train)

# トレーニングデータ
train_sequences = tokenizer.tests_to_sequences(X_train)

# テストデータ
test_sequences = tokenizer.tests_to_sequences(X_test)

In [ ]:
train_sequences

In [ ]:
test_sequences

In [ ]:
print("The encoding for document\n", X_train[1:2],"\n is: ", train_sequences[1])

In [ ]:
# トレーニングとテストにパディングを追加
padded_train = pad_sequences(train_sequences, maxlen = 29, padding = 'post', truncating = 'post')
padded_test = pad_sequences(test_sequences, maxlen = 29, truncating = 'post')

In [ ]:
for i, doc in enumerate(padded_train[:3]):
     print("The padded encoding for document:", i+1," is:", doc)

In [ ]:
# データをカテゴライズされた2次元表現に変換する
y_train_cat = to_categorical(y_train, 2)
y_test_cat = to_categorical(y_test, 2)

In [ ]:
y_train_cat.shape

In [ ]:
y_test_cat.shape

In [ ]:
y_train_cat

**ミニチャレンジ #6:**

- パディングの長さを15に変更して、コードを再実行します。パディングが成功したことを確認する**。


In [ ]:
# トレーニングとテストにパディングを追加する
padded_train = pad_sequences(train_sequences, maxlen = 15, padding = 'post', truncating = 'post')
padded_test = pad_sequences(test_sequences, maxlen = 15, truncating = 'post')

# task #8: リカレントニューラルネットワークと長短記憶ネットワーク(lstm)の理論と直感を理解する

![alt text](https://drive.google.com/uc?id=1Giaz7q1THBFTuNFpSyLBKnoUbbvWlNw3)

![alt text](https://drive.google.com/uc?id=1iDKpQqmGTNr3riuQOvXdiwfy9wlCU5st)

![alt text](https://drive.google.com/uc?id=1PxW6DBer4d1Q9_9OSaAQDTtqUdDGLdYa)

# task #9: センチメント分析を行うカスタムベースのディープニューラルネットワークを構築する

![alt text](https://drive.google.com/uc?id=1zpI1XHM1CSxLPjsW7QTahfs_f2stzKeQ)

In [ ]:
# シーケンシャルモデル
model = Sequential()

# エンベッディング層
model.add(Embedding(total_words, output_dim = 512))

# 双方向RNNとLSTM
model.add(LSTM(256))

# 密な層
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(2,activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

In [ ]:
# モデルの学習
model.fit(padded_train, y_train_cat, batch_size = 32, validation_split = 0.2, epochs = 2)

**ミニチャレンジ #7:**
- **異なるエンベッディング出力次元を用いてモデルをトレーニングする**。


# task #10: 学習したモデルの性能を評価する

![alt text](https://drive.google.com/uc?id=1MZdb0g69XDC4JRATR9K6-2NAkrclGAXO)

In [ ]:
# 予測
pred = model.predict(padded_test)

In [ ]:
# 予測の作成
prediction = []
for i in pred:
  prediction.append(np.argmax(i))

In [ ]:
# 元の値を含むリスト
original = []
for i in y_test_cat:
  original.append(np.argmax(i))

In [ ]:
# テキストデータの精度スコア
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(original, prediction)
accuracy

In [ ]:
# 混同行列のプロット
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(original, prediction)
sns.heatmap(cm, annot = True)

**ミニチャレンジ #8:**

- **事前に学習したBERTモデルを使用して、感情分析の予測を行う**。


# **EXCELLENT JOB!**

**ミニチャレンジ #8:**



**ミニチャレンジ #1 解決策:**

- **「sentiment」列には、いくつのユニークな要素がありますか？2つの異なる方法で調べてみましょう。
sns.countplot(stock_df['Sentiment'])


In [ ]:
sns.countplot(stock_df['Sentiment'])

In [ ]:
# 特定の列に存在するユニークな値の数を求める
stock_df['Sentiment'].nunique()

**MINI CHALLENGE #2 SOLUTION:** 
- **Remove punctuations using a different method**


In [ ]:
fTest_punc_removed = []
for char in Test: 
    if char not in string.punctuation:
        Test_punc_removed.append(char)

# 再び文字を結合して文字列を形成する。  
Test_punc_removed_join = ''.join(Test_punc_removed)
Test_punc_removed_join

**ミニチャレンジ#3の解決策：***。

- 3文字ではなく2文字以上の単語を保持するようにコードを修正する**。
- ストップワードのリストに'https'を追加してコードを再実行する**。


In [ ]:
# nltkから追加のストップワードを取得する
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','will','aap','co','day','user','stock','today','week','year', 'https'])

In [ ]:
# ストップワードを除去し、2文字以下の単語を削除する
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >= 2 and token not in stop_words:
            result.append(token)
            
    return result

**ミニチャレンジ #4 解決策：***。
- **ネガティブな感情を持つツイートのワードクラウドを可視化する**。


In [ ]:
# 否定的な感情を持つテキストのワードクラウドをプロットする
plt.figure(figsize = (20,20)) 
wc = WordCloud(max_words = 1000, width = 1600, height = 800 ).generate(" ".join(stock_df[stock_df['Sentiment'] == 0]['Text Without Punc & Stopwords Joined']))
plt.imshow(wc, interpolation = 'bilinear');

**ミニチャレンジ#5の解決策：***。
- **Seaborn Countplotを使って、肯定的な感情と否定的な感情に属するサンプルの数を視覚的に示します**。


In [ ]:
# 単語数をプロットする
sns.countplot(stock_df['Sentiment'])

**ミニチャレンジ #6 解決策:**

- パディングの長さを15に変更して、コードを再実行します。パディングが成功したことを確認する**。


In [ ]:
# トレーニングとテストにパディングを追加
padded_train = pad_sequences(train_sequences, maxlen = 15, padding = 'post', truncating = 'post')
padded_test = pad_sequences(test_sequences, maxlen = 15, truncating = 'post')

**ミニチャレンジ#7の解決策：***。
- 異なるエンベッディング出力次元を使ってモデルをトレーニングする**。


In [ ]:
model = Sequential()

# エンベッディング層
model.add(Embedding(total_words, output_dim = 256))

# 双方向性RNNとLSTM
model.add(Bidirectional(LSTM(128)))

# 密な層
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])
model.summary()

**ミニチャレンジ #8 解決策：***。

- **事前に学習させたBERTモデルを使用して、感情分析の予測を行う**。


In [ ]:
!pip install transformers

In [ ]:
# 特定のタスクを実行するためにトランスフォーマーからのパイプラインを使用します。
# タスクとしてセンチメント分析を指定し、文字列を渡すと、結果が得られます。
# トピックモデリング、Q&A、テキスト要約などのタスクをここで指定することができます。
from transformers import pipeline

nlp = pipeline('s sentiment-analysis')

# テストデータで予測を行う
pred = nlp(list(X_test))

# 予測値は辞書なので、ラベルをdictから取得する
予測値 = [].
for i in pred:
  prediction.append(i['label'])

# 最終結果を表示する
for i in range(len(prediction[:3])):
  print("˶ˆ꒳ˆ˵", df[df.combined == X_test.values[i]].Text.item(), "˶ˆ꒳ˆ˵", "˶ˆ꒳ˆ˵", "˶ˆ꒳ˆ˵", "˶ˆ꒳ˆ˵",
      category[df[df.combined == X_test.values[i]].Sentiment.item()], "˶nnPredicted value :˶n˶n˶n", prediction[i], "˶n˶n˶n˶n˶n˶n˶n˶")

